In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 66.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=855bc9acca72bb25b32ae515189db1db5dd9d2a00882901c55014a25482fb217
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import time

## Importing Dataset

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- objid: double (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- zerr: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: doub

In [4]:
df.summary()

DataFrame[summary: string, _c0: string, objid: string, modelMag_u: string, modelMag_g: string, modelMag_r: string, modelMag_i: string, modelMag_z: string, fiberMag_u: string, fiberMag_g: string, fiberMag_r: string, fiberMag_i: string, fiberMag_z: string, petroR50_r: string, petroR90_r: string, petroR50_z: string, petroR90_z: string, r: string, i: string, z: string, redshift: string, zerr: string, mmug: string, mmgr: string, mmri: string, mmiz: string, mfug: string, mfgr: string, mfri: string, mfiz: string]

##Data Pre-Processing

In [5]:
df = df.drop('objid', '_c0')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z| redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+---------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|  2

###Check Number of Rows and columns

In [6]:
print(df.count(), len(df.columns))

3524477 27


### Remove Duplicate rows in Dataframe

In [7]:
### Remove Duplicate rows in Dataframe
df = df.dropDuplicates()
print(df.count(), len(df.columns))

3524477 27


Since the number of rows remains the same, this means that there was no duplicate rows in Dataframe

In [8]:
import pyspark.sql.functions as func

df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))

In [9]:
df.groupBy('redshift').count().show()

+--------+-------+
|redshift|  count|
+--------+-------+
|       1| 410758|
|       6|   4287|
|       3|  38810|
|       5|   3321|
|       4|   4902|
|       7|    383|
|       2| 225196|
|       0|2836820|
+--------+-------+



In [10]:
import pandas as pd
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
modelMag_u,21.322890,21.540440,24.099690,19.703640,21.891370
modelMag_g,21.199790,21.344430,21.893860,17.857980,19.901890
modelMag_r,20.978130,20.455390,20.295770,16.785280,18.379160
modelMag_i,20.786010,19.635970,19.441590,16.337970,17.835950
modelMag_z,20.602870,19.201110,19.201700,16.005240,17.496330
fiberMag_u,21.721530,22.420790,24.791400,20.956700,22.894670
fiberMag_g,21.571780,22.167020,22.645580,19.119800,20.870210
fiberMag_r,21.418020,21.318560,21.237360,18.061850,19.368600
fiberMag_i,21.290620,20.557740,20.370210,17.609200,18.812600
fiberMag_z,21.389900,20.050190,20.087840,17.276210,18.468090


In [11]:
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+-

In [12]:
df.columns

['modelMag_u',
 'modelMag_g',
 'modelMag_r',
 'modelMag_i',
 'modelMag_z',
 'fiberMag_u',
 'fiberMag_g',
 'fiberMag_r',
 'fiberMag_i',
 'fiberMag_z',
 'petroR50_r',
 'petroR90_r',
 'petroR50_z',
 'petroR90_z',
 'r',
 'i',
 'z',
 'redshift',
 'zerr',
 'mmug',
 'mmgr',
 'mmri',
 'mmiz',
 'mfug',
 'mfgr',
 'mfri',
 'mfiz']

In [13]:
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+-

In [14]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------+---------+---------+------------------+---------+---------+---------+-----+
|modelMag_u|        modelMag_g|modelMag_r|        modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|petroR50_r|petroR90_r|petroR50_z|       petroR90_z|       r|                 i|                 z|redshift|                zerr|     mmug|     mmgr|     mmri|     mmiz|              mfug|     mfgr|     mfri|     mfiz|objid|
+----------+------------------+----------+------------------+------------------+----------+----------+----------+----------+----------+----------+----------+----------+-----------------+--------+------------------+------------------+--------+--------------------+---------+---------

In [15]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [16]:
output = assembler.transform(df)

In [17]:
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[21.32289,21.1997...|       2|
|[21.54044,21.3444...|       0|
|[24.09969,21.8938...|       0|
+--------------------+--------+
only showing top 3 rows



In [18]:
N = [500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3524477]
time_taken = []
data_point = []

In [19]:
train, test = final_df.randomSplit([0.7, 0.3])

In [20]:
from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier, 
                                      GBTClassifier)
from pyspark.ml import Pipeline

#dt = DecisionTreeClassifier(labelCol = 'redshift', featuresCol = 'features')
rf = RandomForestClassifier(labelCol = 'redshift', featuresCol = 'features')

In [21]:
start_time = time.time()
rf_model = rf.fit(train) 
print("%s seconds" % (time.time() - start_time))

285.10390162467957 seconds


In [22]:
rf_train_prediction = rf_model.transform(train)

In [23]:
rf_train_prediction.show(5)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[13.68416,11.8890...|       0|[19.8083019021244...|[0.99041509510622...|       0.0|
|[13.72725,12.1701...|       0|[19.8083019021244...|[0.99041509510622...|       0.0|
|[13.78523,12.4974...|       0|[19.8083019021244...|[0.99041509510622...|       0.0|
|[14.02077,12.3308...|       0|[19.8083019021244...|[0.99041509510622...|       0.0|
|[14.1087700000000...|       0|[19.8083019021244...|[0.99041509510622...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')

print('Random Forest Training Accuracy:', multi_evaluator.evaluate(rf_train_prediction))
print('Training data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(rf_train_prediction))

Random Forest Training Accuracy: 0.8635029998472422
Training data (weightedPrecision):  0.8635461950769145


In [25]:
rf_test_predictions = rf_model.transform(test)

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Random Forest Test Accuracy:', multi_evaluator.evaluate(rf_test_predictions))

Random Forest Test Accuracy: 0.8636066777849501


In [27]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(rf_test_predictions))
#print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(rf_predictions))

test data (weightedPrecision):  0.8639063514892105


##Cascading

In [28]:
rf_predictions2 = rf_model.transform(final_df)

In [29]:
rf_predictions2.show(3)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[21.32289,21.1997...|       2|[3.90264085368189...|[0.19513204268409...|       1.0|
|[21.54044,21.3444...|       0|[19.6095432309362...|[0.98047716154681...|       0.0|
|[24.09969,21.8938...|       0|[19.7467287445744...|[0.98733643722872...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 3 rows



In [30]:
from pyspark.sql.functions import monotonically_increasing_id
rf_predictions = rf_predictions2.withColumn("objid", monotonically_increasing_id())
rf_predictions.show(3)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[21.32289,21.1997...|       2|[3.90264085368189...|[0.19513204268409...|       1.0|    0|
|[21.54044,21.3444...|       0|[19.6095432309362...|[0.98047716154681...|       0.0|    1|
|[24.09969,21.8938...|       0|[19.7467287445744...|[0.98733643722872...|       0.0|    2|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 3 rows



In [31]:
rf_predictions = rf_predictions.drop('redshift')
rf_predictions.show(4)

+--------------------+--------------------+--------------------+----------+-----+
|            features|       rawPrediction|         probability|prediction|objid|
+--------------------+--------------------+--------------------+----------+-----+
|[21.32289,21.1997...|[3.90264085368189...|[0.19513204268409...|       1.0|    0|
|[21.54044,21.3444...|[19.6095432309362...|[0.98047716154681...|       0.0|    1|
|[24.09969,21.8938...|[19.7467287445744...|[0.98733643722872...|       0.0|    2|
|[19.70364,17.8579...|[19.7812275146883...|[0.98906137573441...|       0.0|    3|
+--------------------+--------------------+--------------------+----------+-----+
only showing top 4 rows



In [32]:
print(df.count(), len(df.columns))
print(rf_predictions.count(), len(rf_predictions.columns))

new_df = df.join(rf_predictions, df.objid == rf_predictions.objid).select(df["*"],rf_predictions["prediction"])
#new_df = df.join(dt_predictions, df.objid == dt_predictions.objid)
new_df.show(5)

3524477 28
3524477 5
+----------+----------+----------+----------+------------------+----------+----------+----------+----------+----------+------------------+----------+------------------+------------------+--------+--------+------------------+--------+-----------+---------+---------+------------------+---------+------------------+---------+---------+--------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|        modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|        petroR90_z|       r|       i|                 z|redshift|       zerr|     mmug|     mmgr|              mmri|     mmiz|              mfug|     mfgr|     mfri|    mfiz|objid|prediction|
+----------+----------+----------+----------+------------------+----------+----------+----------+----------+----------+------------------+----------+------------------+------------------+--------+--------+------------------+--------+-----------+---------+----

In [33]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [34]:
output = assembler.transform(new_df)

In [35]:
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[21.32289,21.1997...|       2|
|[20.66357,20.0293...|       0|
|[21.01588,20.8195...|       1|
+--------------------+--------+
only showing top 3 rows



In [36]:
train, test = final_df.randomSplit([0.7, 0.3])

In [37]:
from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier, 
                                      GBTClassifier)
from pyspark.ml import Pipeline
rf = RandomForestClassifier(labelCol = 'redshift', featuresCol = 'features2')

In [38]:
rf_model2 = rf.fit(train)

In [39]:
rf_train_prediction2 = rf_model2.transform(train)
print('Random Forest Training Accuracy:', multi_evaluator.evaluate(rf_train_prediction2))
print('Training data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(rf_train_prediction2))

Random Forest Training Accuracy: 0.8652378907623469
Training data (weightedPrecision):  0.8652378907623469


In [40]:
rf_predictions2 = rf_model2.transform(test)

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Decision Tree Accu:', multi_evaluator.evaluate(rf_predictions2))

Decision Tree Accu: 0.8655926626098639


In [42]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(rf_predictions2))
#print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(rf_predictions2))

test data (weightedPrecision):  0.8649171781776663


#Before Cascading
Training Accuracy: 0.8635

Training weightedPrecision: 0.8635

Test Accuracy: 0.8636

Test weightedPrecision: 0.8672

#After Cascading
Training Accuracy: 0.8700

Training weightedPrecision: 0.8700

Test Accuracy: 0.8725

Test weightedPrecision: 0.8699

In [ ]:
Random Forest Training Accuracy: 0.8635029998472422
Training data (weightedPrecision):  0.8635461950769145